In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

### Задание. Настроить загрузку данных в Colab

In [0]:
from google.colab import drive # монтируем гугл диск
drive.mount('/content/gdrive')

In [0]:
import os

os.chdir('/content/gdrive/My Drive/Colab Notebooks/Datasets/')  # меняем директорию по умолчанию, куда загружен датасет (у каждого своя)

Дополнительно можно создать директорию прямо в Colab и разархивировать туда, полезно на будущее!
# !mkdir data  # создание директории
# !unzip -q data_classification.zip # разархивирование

### 1. Загрузка данных

### Задание. Попробовать добавить дополнительные transforms: повороты, сдвиги, растяжения и пр.
transforms.RandomRotation(25),
transforms.RandomResizedCrop(224)

https://pytorch.org/docs/stable/torchvision/transforms.html

In [0]:
transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], # нормализация из данных imagenet
                             [0.229, 0.224, 0.225])
    ])
train_set = datasets.ImageFolder("data/train",transforms) # указать правильные директории для закрузки
val_set   = datasets.ImageFolder("data/train",transforms) # указать правильные директории для загрузки
  
train_loader = torch.utils.data.DataLoader(train_set, batch_size=4,
                                       shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=4,  
                                       shuffle=True, num_workers=4)
classes = train_set.classes

device = torch.device("cuda:0" if torch.cuda.is_available() # обратите внимание на использование cuda и gpu
                               else "cpu")

### 2. Fine-tuning 
Используем предобученную на imagenet нейросеть ResNet-34

### Задание. Попробовать модель VGG-16
https://pytorch.org/docs/stable/torchvision/models.html

In [0]:
model = models.resnet34(pretrained=True) 

### 3. Feature extraction
Используем ResNet в качестве экстрактора признаков

In [0]:
model = models.resnet34(pretrained=True)
for param in model.parameters():
    param.requires_grad = False # замораживаем веса, не будем их тренировать!

В ResNet34 последний слой - это FC слой на 1000 нейронов (1000 классов), нам нужна вероятность только по двум классам Cat vs Dog

In [0]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2) # теперь у нас только два класса
model = model.to(device)

criterion = nn.CrossEntropyLoss() # Кросс-энтропийная функция потерь для классификации
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # SGD c моментумом, можно поменять на Adam

### 4. Обучение

In [0]:
for epoch in range(25):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Вспоминаем процесс тренировки в pytorch из предыдущих заданий!
        optimizer.zero_grad() 
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step() 
        
        running_loss += loss.item()
    print(running_loss)
print('Finished Training')

### 5. Валидация

In [0]:
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))
with torch.no_grad():
    for i, data in enumerate(val_loader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

### Задание. Инференс на 5 изображениях из интернета

In [0]:
from PIL import Image
model.eval()
img_name = "1.jpeg" # укажите путь до изображений в своем гугл диске
def predict_image(image_path, model):
    image = Image.open(image_path)
    image_tensor = transforms(image)
    image_tensor = image_tensor.unsqueeze(0)
    image_tensor = image_tensor.to(device)
    output = model(image_tensor)
    index = output.argmax().item()
    if index == 0:
        return "Cat"
    elif index == 1:
        return "Dog"
    else:
        return
predict(img_name,model)